This notebook is to gather PPP data (2021 loans through March 31), join with BLS QCEW data, and calculate PPP penetration for display on ArcGIS as requested by the SBA

Initial settings; Load PPP data

In [1]:
import loans_common2021 as co

import pandas as pd
import zipfile
#from plotnine import *    # python lib to use ggplot
from io import BytesIO
from urllib.request import urlopen

pd.set_option('max_columns', 100)

naics = 'NAICS3'
excsole = True

fpath = '/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/'\
                                                    'FOIA 2021-04-01/'
loansNE = co.ReadPPPdata(naics, fpath)

/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_9.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_8.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_150k_plus.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_3.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_2.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_1.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_5.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_4.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k_6.csv
/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/FOIA 2021-04-01/public_up_to_150k

## Exclude loans to sole proprietors, contractors etc

In [2]:
soleprop = ['Independent Contractors' #,'Self-Employed Individuals'
            ,'Sole Proprietorship', 'Tenant in Common']
loans = loansNE[~loansNE.BusinessType.isin(soleprop)]

## Join County info into each loan 

In [3]:
[loans_y, countyfips] = co.MatchCounties(loans, excsole, naics)

unique cities in loan dataset: 3085
Unique matches: 1405
Unique non-matches: 1680


/Users/aligo/git-repos/FEMA/loans_common2021.py:182: FutureWarning: The default value of regex will change from True to False in a future version.
  citycounty['cityc'] = citycounty['NAMELSAD10'].str.replace(pat, '').str.upper().str.strip()


loans in loan dataset: 325758
loans with matches: 283427
loans non-matches: 42331
zip-county pairs in New Englad: 2496
zip-county pairs in New England, deduplicated: 2206
loans in loan dataset: 325758
1st draw loans:  229810
2nd draw loans:  95948
After ZIP match: loans with matches:  325733
After ZIP match: loans non-matches:  25
MANUALLY FIX THE COUNTIES IN EXCEL FILE /Users/aligo/Downloads/FEMA recovery data/ppp addresses MISSnosole2021.xlsx
 AND CHANGE ITS NAME FROM MISS TO MANUAL


### EXECUTE THIS COMMAND ONLY AFTER CHECKING THE EXCEL FILE OF UNMATCHED LOANS in Downloads/

In [4]:
loansc = co.AddManualCounties(loans_y, excsole, naics)

loans in loan dataset: 325758
loans with matches: 325733
loans non-matches: 25


## TOTAL NUMBER OF BUSINESSES - read total number of businesses from BLS

In [5]:
fpath = '/Users/aligo/Downloads/FEMA recovery data/'
zf = zipfile.ZipFile( fpath + '2020_qtrly_singlefile.zip') 
qcew = pd.read_csv(zf.open('2020.q1-q2.singlefile.csv'), dtype={'area_fips':'object'})
# filter
idx = (qcew.own_code.eq(5) & qcew.qtr.eq(1) & # private, FIRST quarter only
         qcew['area_fips'].isin(countyfips['COUNTY'].tolist()))   # NE only
if (naics == 'NAICS2'):
    idx = idx & ((qcew['industry_code'].str.len().eq(2)   # 2-digit NAICS
            | qcew['industry_code'].str.match('\d\d-\d\d'))    # 2-digit NAICS that are joint, e.g. NAICS 31-33 Manufacturing
          & ~qcew['industry_code'].eq('10') )      # exclude code = 10 that are "ALL Naics" totals
else:
    idx = idx & (qcew['industry_code'].str.len().eq(3)   # 3-digit NAICS
          & ~qcew['industry_code'].str.startswith('10') )      # exclude code = 10 that are "ALL Naics" totals
    
cnt = qcew[idx]  # New England counties
cnt = cnt[['area_fips','industry_code','qtrly_estabs']]
# Add state and county name
cnt = cnt.set_index('area_fips').join(countyfips.set_index('COUNTY')).reset_index()
# Join Total Num businesses + Num of loans
cnt.columns = ['COUNTYfips',naics,'NEstabs','State','COUNTYName']

### Subset scope of loans draws and calculate penetration; change from 1st draw (2020+2021) to 2nd draw

In [6]:
pen = co.CalcPenforScopes(loansc, naics, cnt)

*** First draw (2020 and 2021) ***
Total Number of PPP loans to New England excluding loans with missing info: 228353.0
Total Count of businesses in NE: 496993
Total Number of County-NAICS pairs in NE with existing businesses: 5221
Previous Number with penetration > 1: 654
Previous Number with penetration = 0: 8
*** First draw (2020 only) ***
Total Number of PPP loans to New England excluding loans with missing info: 210651.0
Total Count of businesses in NE: 496993
Total Number of County-NAICS pairs in NE with existing businesses: 5221
Previous Number with penetration > 1: 530
Previous Number with penetration = 0: 8
*** Second draw (2021) ***
Total Number of PPP loans to New England excluding loans with missing info: 95460.0
Total Count of businesses in NE: 496993
Total Number of County-NAICS pairs in NE with existing businesses: 5221
Previous Number with penetration > 1: 65
Previous Number with penetration = 0: 0
*** Both draws (2021) ***
Total Number of PPP loans to New England exclu

In [114]:
pen.to_csv( '/Users/aligo/Downloads/FEMA recovery data/PPP_loans_from_SBA/PPPpenetration_2021_BLS_' + naics + '.csv' )

In [7]:
# clean spaces and uppercase business names, addresses
loansc['BorrowerNameC'] = loansc['BorrowerName'].str.upper().str.strip()
loansc['BorrowerAddressC'] = loansc['BorrowerAddress'].str.upper().str.strip()
loansc['BorrowerCityC'] = loansc['BorrowerCity'].str.upper().str.strip()
loansc['Zip5'] = loansc['BorrowerZip'].str.slice(0,5) # first 5 digits of zip

# cols to use to uniquely identify loan
cols = ['BorrowerNameC','BorrowerAddressC','BorrowerCityC','COUNTYfips','StateAbb','Zip5']

# check uniqueness of business name + full address
def CheckDuplic(loans, ProcessingMethod):
    tmp = loans[loans['ProcessingMethod']==ProcessingMethod]
    u = len(tmp[cols].drop_duplicates())
    print( f'{ProcessingMethod} draw has {tmp.shape[0]:,} loans with {u:,} unique BorrowerName+ full addr' )

    # save duplicates - (TO DO: ASK SBA)
    tmp[tmp.duplicated(cols,keep=False)].sort_values(by=['BorrowerNameC']).to_excel(f'/Users/aligo/Downloads/tmp/{ProcessingMethod}_DUPLICATES.xlsx')

    # drop duplicates
    return tmp.drop_duplicates(subset=cols).sort_values(by=['BorrowerNameC'])

loans_draw1 = CheckDuplic(loansc, 'PPP')
loans_draw2 = CheckDuplic(loansc, 'PPS')

PPP draw has 229,787 loans with 229,721 unique BorrowerName+ full addr
PPS draw has 95,946 loans with 95,934 unique BorrowerName+ full addr


In [25]:
# join loan draws by exact match of 1st and 2nd rounds
tmp = loans_draw1.set_index(cols).join(loans_draw2.set_index(cols), how='outer', lsuffix='_1', rsuffix='_2', sort=False)
loans_matched = tmp[(tmp['LoanNumber_1'].notna()) & (tmp['LoanNumber_2'].notna())]
print( f'2nd draw loans total {loans_draw2.shape[0]:,}, of which {loans_matched.shape[0]:,} find a 1st draw byt an exact match of name and address' )
print( f'join consistent? {(loans_draw1.shape[0] + loans_draw2.shape[0] - loans_matched.shape[0]) == tmp.shape[0]}')
idx = loans_matched['LoanNumber_1'].tolist() + loans_matched['LoanNumber_2'].tolist()
tmp = pd.concat([loans_draw1, loans_draw2])
loans_unmatched = tmp[~(tmp.set_index('LoanNumber').index.isin(idx))].reset_index()

2nd draw loans total 95,934, of which 17,162 find a 1st draw byt an exact match of name and address
join consistent? True


### Match draws going to the same business

In [67]:
features = co.MatchLoanDraws(loans_unmatched)   # matches loans using Probabilistic record linkage with Business name and address

ProcessingMethod ['PPP' 'PPS']


/Users/aligo/opt/anaconda3/lib/python3.7/site-packages/recordlinkage/preprocessing/cleaning.py:114: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'(\[.*?\]|\(.*?\)|\{.*?\})', '')
/Users/aligo/opt/anaconda3/lib/python3.7/site-packages/recordlinkage/preprocessing/cleaning.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_none, '')
/Users/aligo/opt/anaconda3/lib/python3.7/site-packages/recordlinkage/preprocessing/cleaning.py:121: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(replace_by_whitespace, ' ')
/Users/aligo/opt/anaconda3/lib/python3.7/site-packages/recordlinkage/preprocessing/cleaning.py:124: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.replace(r'\s\s+', ' ')


212,559 1st draws and 78,772 second draws to match
MatchLoanDraws for block ['StateAbb', 'BorrowerCityC'] at 17:13:45; Number of pairs to match: 60,091,622
MatchLoanDraws finished at 17:21:44


In [83]:
# features.sum(axis=1).value_counts().sort_index(ascending=False)
fscores = features.assign( score = features.sum(axis=1)
                         , match = -1 ) # not visited
fscores = fscores.sort_index()
for n2 in fscores.index.get_level_values(1).unique():
    print( n2 )
    n2df = fscores.loc[(slice(None), slice(n2)) & fscores['match']==-1,:]
    nsdf['match'] = 0 # visited
    max_score = nsdf.max()
    idx = nsdf[nsdf['score']==max_score]
    pen4.loc[pen4['color'] > 1,'color'] = 1
    break

1003808408


In [28]:
loansmatch.to_csv('/Users/aligo/Downloads/FEMA recovery data/PPPloansNE2021-MatchedDraws.csv')

In [ ]:
# total per county 
dfc = pen.groupby(['COUNTYName']).agg('sum').reset_index()
dfc['penetration'] = dfc['NLoans'] / dfc['NEstabs']
ggplot(dfc, aes(x='reorder(COUNTYName,penetration)', y='penetration')
          ) + geom_bar(stat="identity"
          ) + xlab('County'
          ) + ylab('PPP Loan Penetration'
          ) + ggtitle('New England PPP Penetrations per County'
          ) + theme(axis_text_y = element_text(size=6)
          ) + coord_flip()

# total per NAICS
dfp = pen.groupby(['NAICS2','NAICSdescr']).agg('sum').reset_index()
dfp['penetration'] = dfp['NLoans'] / dfp['NEstabs']

ggplot(dfp, aes(x='reorder(NAICSdescr,penetration)', y='penetration')
          ) + geom_bar(stat="identity"
          ) + xlab('NAICS 2 Digit Sector'
          ) + ylab('PPP Loan Penetration'
          ) + ggtitle('New England PPP Penetrations per NAICS Sector'
#          ) + scale_y_continuous(trans = 'log2'
          ) + coord_flip()

# Histogram of County-NAICS penetrations
ggplot(pen[pen.penetration.le(1)], aes(x='penetration')
       ) + geom_histogram(binwidth=.05
       ) + xlab('PPP Loan Penetration'
       ) + ylab('Number of Counties-Sectors'
       ) + ggtitle('Distribution of PPP Penetration <= 1 in New England')

# outlier penetrations
ggplot(pen, aes(x='State', y='penetration')
       ) + geom_boxplot(
       ) + xlab('State'
       ) + ylab('Penetration'
       ) + ggtitle('Outliers of PPP Penetration in New England')





